In [32]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd #for reading csv files
from sklearn.model_selection import GridSearchCV  # Import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

In [33]:
training=pd.read_csv('training_data.csv', sep=',')
test=pd.read_csv('songs_to_classify.csv', sep=',')

only use five features version

In [34]:
# select which features to use
features = ['danceability','key','loudness','instrumentalness','liveness']

# i learn how to do one-hot encode from chatgpt because i realized there is no size meaning in the key feature.  
encoder = OneHotEncoder(sparse=False)

X_train_selected = training[features]
X_test_selected = test[features]
# OneHotEncode the 'key' feature 
X_train_key_encoded = encoder.fit_transform(X_train_selected[['key']])
X_test_key_encoded = encoder.transform(X_test_selected[['key']])

X_train_no_key = X_train_selected.drop('key', axis=1).values
X_test_no_key = X_test_selected.drop('key', axis=1).values

X_train = np.hstack([X_train_no_key, X_train_key_encoded])
X_test = np.hstack([X_test_no_key, X_test_key_encoded])

# extract labels
y_train = training.loc[:, 'label'].values

In [35]:
# Normalize data. Can also be done using sklearn methods such as
# MinMaxScaler()
X_trainn = X_train*1/np.max(np.abs(X_train),axis=0)
X_testn = X_test*1/np.max(np.abs(X_train),axis=0)

using all features version

In [36]:
# i learned how to choose all the features from chatgpt but now i think i could figure it out myself

#encoder = OneHotEncoder(sparse=False)
#X_train_key_encoded = encoder.fit_transform(X_train_selected[['key']])
#X_test_key_encoded = encoder.transform(X_test_selected[['key']])


# Remove the 'key' column and keep other features
#X_train_no_key_time = training.loc[:, training.columns != 'key'].drop('label', axis=1).values
#X_test_no_key_time = test.loc[:, test.columns != 'key'].values

# Merge OneHot encoded 'key' with other features
#X_train = np.hstack([X_train_no_key_time, X_train_key_encoded, X_train_time_signature_encoded])
#X_test = np.hstack([X_test_no_key_time, X_test_key_encoded, X_test_time_signature_encoded])

# Extract labels
#y_train = training.loc[:, 'label'].values

In [37]:
# note: all inputs/features are treated as quantitative/numeric
# some of the features are perhaps more sensible to treat as
# qualitative/cathegorical. For that sklearn preprocessing methods
# such as OneHotEncoder() can be used

# Define the parameter grid for n_neighbors
knnmodel = KNeighborsClassifier(n_neighbors = 5)

define the k-NN model. To set n_neighbors in a systematic way, use cross validation!

In [38]:
# Define the parameter grid for n_neighbors

# i use chatgpt to generate this code for choosing these hyperparameters using GridSearchCV. 
# I know this method but i was not sure how to implement it in code.
param_grid = {'n_neighbors': range(1, 21)}  # Try n_neighbors from 1 to 20

# Use GridSearchCV to find the best n_neighbors
grid_search = GridSearchCV(estimator=knnmodel, param_grid=param_grid, cv=5)  # 5-fold cross-validation
grid_search.fit(X=X_trainn, y=y_train)

# Get the best n_neighbors from the grid search
best_n_neighbors = grid_search.best_params_['n_neighbors']
print(f"Best n_neighbors: {best_n_neighbors}")

Best n_neighbors: 7


In [39]:
# Train the k-NN model with the best n_neighbors
knnmodel_best = KNeighborsClassifier(n_neighbors=best_n_neighbors)
knnmodel_best.fit(X=X_trainn, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [40]:
# Make predictions
# Chagpt designed this output format for me and i asked it to remove spaces between elements so that i could use it for submission
predictions = knnmodel_best.predict(X=X_testn).reshape(-1, 1).astype(int).reshape(1, -1)
#print(predictions)
print(''.join(map(str, predictions.flatten())))

00010001101001111011001000110110111111011001110111111101110011001101110011010110111001111110001111100010110011100101000001101110111111111110001111001111101111100100101111111010111001111100011010111111


Cross Validation

In [41]:
# i use chatgpt to do cross validation because I need a standard to evaluate the pros and cons of different models
cv_scores_lr = cross_val_score(knnmodel_best, X_trainn, y_train, cv=5, scoring='accuracy')
print(f'KNN cross-validation accuracy: {cv_scores_lr.mean():.4f} ± {cv_scores_lr.std():.4f}')

KNN cross-validation accuracy: 0.7267 ± 0.0280
